In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from mealpy.swarm_based import GWO

import torch
import torch.nn as nn
import torch.optim as optim

# Carregamento dos dados

In [2]:
data_set_unformatted = pd.read_csv("./CompiledMeanDataRounded.csv", decimal=",", parse_dates=["Data"], infer_datetime_format=True).drop_duplicates()
data_set_unformatted.head()

,Data,% Iron Feed,% Silica Feed,Starch Flow,Amina Flow,Ore Pulp Flow,Ore Pulp pH,Ore Pulp Density,Flotation Column 01 Air Flow,Flotation Column 02 Air Flow,...,Flotation Column 07 Air Flow,Flotation Column 01 Level,Flotation Column 02 Level,Flotation Column 03 Level,Flotation Column 04 Level,Flotation Column 05 Level,Flotation Column 06 Level,Flotation Column 07 Level,% Iron Concentrate,% Silica Concentrate
0,2017-03-29 12:00:00,60.18,9.35,1063.45,379.6,400.97,9.53,1.55,200.04,195.57,...,249.99,755.38,728.0,862.04,477.46,452.56,478.22,469.82,66.45,1.37
1,2017-03-29 13:00:00,60.18,9.35,2032.86,322.95,400.47,9.71,1.54,199.99,195.13,...,250.07,848.67,777.59,869.45,483.95,469.64,471.67,462.91,66.58,1.43
2,2017-03-29 14:00:00,60.18,9.35,1426.1,474.89,399.15,9.69,1.65,200.03,195.58,...,250.11,852.0,776.32,879.69,455.97,453.18,447.52,453.63,66.64,1.33
3,2017-03-29 15:00:00,60.18,9.35,616.73,395.13,398.94,9.86,1.56,199.94,195.61,...,250.04,855.89,780.4,882.1,449.4,448.52,450.87,448.41,66.4,1.3
4,2017-03-29 16:00:00,59.54,9.56,1369.26,318.08,400.82,9.94,1.54,199.89,196.25,...,250.12,851.58,784.47,884.86,450.22,451.9,451.58,449.39,63.65,5.48


In [3]:
data_set_unformatted = data_set_unformatted.to_numpy()
print(data_set_unformatted.shape)

(3948, 24)


## Normalização dos dados

In [4]:
data_set_unformatted = data_set_unformatted[:,1:-1]
print(data_set_unformatted.shape)

(3948, 22)


In [5]:
data_set_unformatted = np.array(data_set_unformatted, dtype=np.float32)

In [6]:
data_set = np.empty(data_set_unformatted.shape)
for i in range(data_set_unformatted.shape[1]):
    tag_max = np.max(data_set_unformatted[:, i])
    tag_min = np.min(data_set_unformatted[:, i])
    scope = tag_max - tag_min
    
    for j in range(data_set_unformatted.shape[0]):
        data_set[j, i] = (data_set_unformatted[j, i] - tag_min)/scope

# Divisão do dados em treinamento e teste

In [7]:
# Separando em conjunto de treinamento e teste
training_data = data_set[:2775,:]
test_data = data_set[2769:,:]

print(f"Training Samples: {training_data.shape[0] - 6}")
print(f"Test Samples: {test_data.shape[0] - 6}")

Training Samples: 2769
Test Samples: 1173


In [8]:
FEATURES_INPUT = data_set.shape[1] * 6
print(FEATURES_INPUT)

132


In [9]:
training_input = np.empty((training_data.shape[0] - 6, FEATURES_INPUT), dtype=np.float32)
training_label = np.empty((training_data.shape[0] - 6, 1), dtype=np.float32)

for i in range(6, training_data.shape[0]):
    training_input[i - 6,:] = training_data[i:i-6:-1,:].reshape(FEATURES_INPUT)
    training_label[i - 6] = training_data[i, 21]

print(training_input.shape)
print(training_label.shape)

training_input = torch.from_numpy(training_input)
training_label = torch.from_numpy(training_label)

(2769, 132)
(2769, 1)


In [10]:
test_input = np.empty((test_data.shape[0] - 6, FEATURES_INPUT), dtype=np.float32)
test_label = np.empty((test_data.shape[0] - 6, 1), dtype=np.float32)

for i in range(6, test_data.shape[0]):
    test_input[i - 6,:] = test_data[i:i-6:-1,:].reshape(FEATURES_INPUT)
    test_label[i - 6]   = test_data[i, 21]

test_input = torch.from_numpy(test_input)
test_label = torch.from_numpy(test_label)

test_input = torch.transpose(test_input, 0, 1)
test_label = torch.transpose(test_label, 0, 1)

print(test_input.size())
print(test_label.size())

torch.Size([132, 1173])
torch.Size([1, 1173])


# Implementação do Modelo

In [13]:
class LSTMPredictor(nn.Module):
    def __init__(self, hidden_layers, n_hidden, input_size):
        super(LSTMPredictor, self).__init__()
        self.hidden_layers = hidden_layers
        self.input_size = input_size
        self.n_hidden = n_hidden

        self.lstm_input = nn.LSTMCell(self.input_size, self.n_hidden )
        self.lstm_list = [nn.LSTMCell(self.n_hidden , self.n_hidden ) for i in range(self.hidden_layers)]
        self.linear = nn.Linear(self.n_hidden , 1)

    def forward(self, input):
        outputs = []
        n_samples = 1


        h_t = torch.zeros(n_samples, self.n_hidden , dtype=torch.float32)
        c_t = torch.zeros(n_samples, self.n_hidden , dtype=torch.float32)
        h_t_list = [torch.zeros(n_samples, self.n_hidden , dtype=torch.float32) for i in range(self.hidden_layers)]
        c_t_list = [torch.zeros(n_samples, self.n_hidden , dtype=torch.float32) for i in range(self.hidden_layers)]

        for input_t in input.split(1, dim=1):
            input_t = torch.transpose(input_t, 0, 1)
            h_t, c_t = self.lstm_input(input_t, (h_t, c_t))
            print(h_t)

            for i in range(self.hidden_layers):
                h_in = None
                if i == 0:
                    h_in = h_t
                else:
                    h_in = h_t_list[i-1]

                print(h_in)

                h_t_list[i], c_t_list[i] = self.lstm_list[i](h_in, (h_t_list[i], c_t_list[i]))
            
            h_t_out = h_t_list[-1]
            output = self.linear(h_t_out)
            outputs.append(output)
        
        outputs = torch.cat(outputs, dim=1)
        return outputs

# Cria e executa Modelo

In [14]:
hidden_layers, hidden_size, learning_rate = 2, 30, 0.001

print("################################################################")
print(f"Hidden Layers: {hidden_layers} | Hidden Size: {hidden_size} | Learning Rate: {learning_rate}")
print("################################################################")

HIDDEN_SIZE = hidden_size
model = LSTMPredictor(hidden_layers, HIDDEN_SIZE, FEATURES_INPUT)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

def train(line_tensor, category_tensor):
    line_tensor = torch.transpose(line_tensor, 0, 1)
    category_tensor = torch.transpose(category_tensor, 0, 1)

    out = model(line_tensor)
    loss = criterion(out, category_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

batch_size = 32

def random_sample():
    start = np.random.randint(training_input.shape[0] - batch_size)
    return training_input[start:start+batch_size,:], training_label[start:start+batch_size,:]

n_epochs = 0
max_epochs = 50
n_batches = 42
desirable_loss = 0.003

new_loss = 1

all_losses = []

for i in range(max_epochs):
    n_epochs += 1
    
    for k in range(n_batches):
        input_batch, category_batch = random_sample()

        train(input_batch, category_batch)

    with torch.no_grad():
        out = model(test_input)
        loss = criterion(out, test_label)
        new_loss = loss.item()

        print(f"epoch {i+1}: {new_loss}")
        all_losses.append(new_loss)

        if new_loss < desirable_loss:
            break
print()
print(f"Final Loss: {new_loss}  | N_Epochs: {n_epochs}")
print()
print()

################################################################
Hidden Layers: 2 | Hidden Size: 30 | Learning Rate: 0.001
################################################################
tensor([[-0.3375, -0.0462, -0.1114, -0.0031,  0.0514,  0.2868, -0.0452, -0.0014,
         -0.0997,  0.0952,  0.0704,  0.1434,  0.0678, -0.0705, -0.0087, -0.0354,
          0.0956, -0.1884, -0.1840,  0.1041, -0.1757,  0.1789, -0.0327, -0.1682,
         -0.0026,  0.1024, -0.0457, -0.0487, -0.0618, -0.0378]],
       grad_fn=<MulBackward0>)
tensor([[-0.0497, -0.0697, -0.0075,  0.0163, -0.0749, -0.0097,  0.0475, -0.0436,
         -0.0098, -0.0034, -0.0648,  0.0451, -0.0038, -0.0007,  0.0039,  0.0106,
         -0.0296, -0.0569, -0.0062, -0.0559,  0.0537,  0.0252,  0.0138,  0.0405,
         -0.0313, -0.0379,  0.0068,  0.0427, -0.0259, -0.0283]],
       grad_fn=<MulBackward0>)
tensor([[-0.4776, -0.1072, -0.1182, -0.0025,  0.1089,  0.3906, -0.0572, -0.0099,
         -0.1530,  0.1663,  0.1347,  0.1605,  0.0875,

KeyboardInterrupt: 